In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [4]:
data = pd.read_csv('../../init_data.csv', header=0, index_col=0)
data

,StateCode,State,nb,nb_idx,Available capacity
0,AK,Alaska,['WA'],[46],200
1,AL,Alabama,"['FL', 'GA', 'MS', 'TN']","[8, 9, 24, 41]",1344
2,AR,Arkansas,"['LA', 'MO', 'MS', 'OK', 'TN', 'TX']","[17, 23, 24, 35, 41, 42]",500
3,AZ,Arizona,"['CA', 'CO', 'NM', 'NV', 'UT']","[4, 5, 31, 32, 43]",1500
4,CA,California,"['AZ', 'HI', 'NV', 'OR']","[3, 10, 32, 36]",11036
5,CO,Colorado,"['AZ', 'KS', 'NE', 'NM', 'OK', 'UT', 'WY']","[3, 15, 28, 31, 35, 43, 49]",600
6,CT,Connecticut,"['MA', 'NY', 'RI']","[18, 33, 38]",1000
7,DE,Delaware,"['MD', 'NJ', 'PA']","[19, 30, 37]",400
8,FL,Florida,"['AL', 'GA']","[1, 9]",4000
9,GA,Georgia,"['AL', 'FL', 'NC', 'SC', 'TN']","[1, 8, 26, 39, 41]",1006


In [5]:
idx = 0
flow_mapping = {}
flow_mapping_rev = {}
out_flow = defaultdict(list)
in_flow = defaultdict(list)

In [6]:
for i in range(51):
    nbs = eval(data.iloc[i, 3])
    out_flow[i] = nbs
    for n in nbs:
        flow_mapping[(i, n)] = idx
        flow_mapping_rev[idx] = (i, n)
        in_flow[n].append(i)
        idx += 1

In [7]:
out_flow[51] = list(range(51))
for n in range(51):
    flow_mapping[(51, n)] = idx
    flow_mapping_rev[idx] = (i, n)
    idx += 1

In [8]:
idx

273

In [19]:
ini = data['Available capacity']

In [20]:
# Farmer: Annotated with location of stochastic matrix entries
#         for use with pysp2smps conversion tool.
#
# Imports
#
from pyomo.core import *
from pyomo.pysp.annotations import StochasticConstraintBoundsAnnotation
from pyomo.environ import RangeSet
#
# Model
#

model = ConcreteModel()

#
# Sets
#
T = 14
S = 51
F = idx
model.time_set = list(range(1, T + 1))
model.state_set = list(range(S))
model.state_p_set = list(range(S + 1))
model.flow_set = list(range(F))
#
# Parameters
#
model.init = ini
model.demand = Param(model.state_set, model.time_set, within=NonNegativeReals, mutable=True)
model.U = [0.1 * model.init[flow_mapping_rev[j][0]] for j in model.flow_set]

In [21]:
# first-stage variables
model.s0 = Var(model.state_p_set, within=NonNegativeReals)
model.x = Var(model.flow_set, within=NonNegativeReals)

# second-stage variables
model.delta = Var(model.state_set, model.time_set, within=NonNegativeReals)
model.theta = Var(model.state_set, within=NonNegativeReals)
model.mu = Var(model.state_set, model.time_set, within=NonNegativeReals)

# always define the objective as the sum of the stage costs
model.FirstStageCost = 0
model.SecondStageCost = \
    Expression(initialize=(sum(model.delta[s, t] for s in model.state_set for t in model.time_set)))

# always define the objective as the sum of the stage costs
model.obj = Objective(expr=model.FirstStageCost + model.SecondStageCost)


In [35]:
# first stage constraints
model.resourse = model.Constraint(model.state_p_set, \
                                  rule=lambda model, j: model.s0[j] - sum(model.x[flow_mapping[(i, j)]] for i in in_flow[j]) + sum(model.x[flow_mapping[(j, i)]] for i in out_flow[j]) == model.init[j])
model.x_bound = model.Constraint(model.flow_set, \
                                  rule=lambda model, j: model.x[j] <= model.U[j])


In [38]:
# second stage constraints
model.s_theta = model.Constraint(model.state_set, \
                                  rule=lambda model, j: model.s0[j] - model.theta[j] >= 0)
model.theta_a = model.Constraint(model.state_set, \
                                  rule=lambda model, j: model.theta[j] <= model.init[j])
model.theta_mu = model.Constraint(model.state_set, \
                                  rule=lambda model, j: model.theta[j] - model.mu[j] <= 0)
model.theta_mu = model.Constraint(model.state_set, \
                                  rule=lambda model, j: model.theta[j] - model.mu[j] <= 0)

# 
# these two constraints have stochastic right-hand-sides
#
model.mu_d = model.Constraint(model.state_set, model.time_set, \
                                  rule=lambda model, j, t: model.mu[j] >= model.demand[j, t])
model.delta_s_d = model.Constraint(model.state_set, model.time_set, \
                                  rule=lambda model, j, t: model.delta[j, t] + model.s0[j] >= model.demand[j, t])

In [42]:
model.demand_table = {}
for j in model.state_set:
    for t in model.time_set:
        model.demand_table[j, t] = [0]

[]

In [ ]:
from pyomo.pysp.annotations import StochasticConstraintBoundsAnnotation
from basemodel import model
model.stoch_rhs = StochasticConstraintBoundsAnnotation()
model.stoch_rhs.declare(model.demand)

num_scenarios = 1
scenario_data = {'Scenario1' : tuple(model.demand_table[j, t][0] for j in model.state_set for t in model.time_set)}

In [56]:
model.demand[0, 1].value = 1

{'Scenario1': (0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  